In [95]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
!pip install datasets

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.datasets import load_files
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from datasets import load_dataset
import os, os.path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [98]:
train_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/train'
test_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/test'
val_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/val'

labels = ['Cam thao', 'Cay bac ha', 'Cay luu ly', 'Cay mac co', 'Cay ma de', 'Co thom', 'Dinh huong', 'Dong trung ha thao', 'Hat de ngua', 'Hoa dau biec', 'Ich mau', 'Ngai cuu', 'Nha dam', 'Toi', 'Trinh nu hoang cung', 'Y di']

In [99]:
def load_image(data_dir):
    dataset = load_dataset("imagefolder", data_dir = data_dir)
    images = dataset['train']['image']
    labels = dataset['train']['label']
    return images, labels

In [100]:
X_train, y_train = load_image(train_data_dir)
X_test, y_test = load_image(test_data_dir)
X_val, y_val = load_image(val_data_dir)

Resolving data files:   0%|          | 0/3924 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

In [101]:
def one_hot_coding(y):
    # Create a one-hot encoder
    encoder = OneHotEncoder()

    # Fit and transform the data
    one_hot_data = encoder.fit_transform(np.array(y).reshape(-1, 1)).toarray()

    return one_hot_data

def decode_one_hot(one_hot_data, labels):
     # Get the index of the non-zero element in each row
    decoded_data_to_int = np.argmax(one_hot_data, axis=1)

    # Assign the corresponding unique values
    decoded_data_to_origin = [labels[i] for i in decoded_data_to_int]

    return decoded_data_to_int, np.array(decoded_data_to_origin)

In [102]:
y_train = one_hot_coding(y_train)
y_test = one_hot_coding(y_test)
y_val = one_hot_coding(y_val)

In [103]:
def convert_image_to_array_form(X):
    image_array = []
    for image in X:
        image = np.array(image)
        new_size = (224, 224)
        img_resized = cv2.resize(image, new_size)
        image_array.append(img_resized)
    return np.array(image_array)

In [104]:
X_train = convert_image_to_array_form(X_train)
X_test = convert_image_to_array_form(X_test)
X_val = convert_image_to_array_form(X_val)

In [105]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.3),

    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.3),


    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.4),

    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.4),

    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(256, activation='relu'),  # Giảm kích thước lớp Dense
    keras.layers.Dropout(0.5),

    keras.layers.Dense(128, activation='relu'),  # Giảm kích thước lớp Dense
    keras.layers.Dropout(0.5),

    keras.layers.Dense(16, activation='softmax')
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'Recall']
)

In [106]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 batch_normalization_28 (Ba  (None, 224, 224, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 112, 112, 32)      0         
 ng2D)                                                           
                                                                 
 dropout_22 (Dropout)        (None, 112, 112, 32)      0         
                                                                 
 conv2d_29 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_29 (Ba  (None, 112, 112, 64)     

In [ ]:
history = model.fit(X_train,y_train, batch_size = 32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
123/123 [==============================] - 29s 144ms/step - loss: 7.2728 - accuracy: 0.0627 - recall: 0.0238 - val_loss: 2.6923 - val_accuracy: 0.1458 - val_recall: 0.0000e+00
Epoch 2/150
123/123 [==============================] - 17s 139ms/step - loss: 2.8353 - accuracy: 0.0676 - recall: 0.0020 - val_loss: 2.7379 - val_accuracy: 0.0781 - val_recall: 0.0000e+00
Epoch 3/150
123/123 [==============================] - 17s 137ms/step - loss: 2.7663 - accuracy: 0.0806 - recall: 0.0051 - val_loss: 2.6957 - val_accuracy: 0.0990 - val_recall: 0.0000e+00
Epoch 4/150
123/123 [==============================] - 17s 136ms/step - loss: 2.7273 - accuracy: 0.0850 - recall: 0.0128 - val_loss: 2.6372 - val_accuracy: 0.1615 - val_recall: 0.0156
Epoch 5/150
123/123 [==============================] - 17s 137ms/step - loss: 2.7466 - accuracy: 0.0834 - recall: 0.0151 - val_loss: 2.7968 - val_accuracy: 0.1198 - val_recall: 0.0208
Epoch 6/150
123/123 [==============================] - 17s 138ms/ste

In [ ]:
model_path = '/content/drive/MyDrive/Thoa/v5.h5'
model.save(model_path)

In [ ]:
import matplotlib.pyplot as plt
#plotting the Accuracy of test and training sets
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/content/drive/MyDrive/Thoa/v5_acc.png", dpi=300)
plt.show()

In [ ]:
#plotting the loss of test and training sets
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/content/drive/MyDrive/Thoa/v5_loss.png", dpi=300)
plt.show()

In [ ]:
acc_score = model.evaluate(X_train, y_train) #we are starting to test the model here
print('\n', 'Train accuracy:', acc_score[1])

acc_score = model.evaluate(X_val, y_val) #we are starting to test the model here
print('\n', 'Val accuracy:', acc_score[1])

acc_score = model.evaluate(X_test, y_test) #we are starting to test the model here
print('\n', 'Test accuracy:', acc_score[1])

In [ ]:
predictions = model.predict(X_test)
fig = plt.figure(figsize=(16, 9))
for i, idx in enumerate(np.random.choice(X_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test[idx]))
    pred_idx = np.argmax(predictions[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(labels[pred_idx], labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))